In [47]:
#Se quiere importar un archivo pickle con un diccionario y transformarlo en un dataframe de pandas

import pandas as pd
import pickle
import re
#Se importa el archivo pickle
with open('scrap_info_alq.pkl', 'rb') as f:
    diccionario = pickle.load(f)


with open('scrap_info.pkl', 'rb') as f:
    diccionario2 = pickle.load(f)
#Se transforma el diccionario en un dataframe
df = pd.DataFrame(diccionario)
df2 = pd.DataFrame(diccionario2)
#Ahora se cambia las filas por las columnas
df = df.T
df2 = df2.T
df.head()
df2.head()

#Ahora unimos los dos dataframes
df_def = pd.concat([df,df2])
#Limpiamos la columna de precio quitando el simbolo de € y quitando el /mes

df_def['Precio'] = df_def['Precio'].str.replace('€','')
df_def['Precio'] = df_def['Precio'].str.replace('/mes','')

#transformamos el el precio a numerico	
df_def['Precio'] = df_def['Precio'].str.replace('.','')

#Eliminamos la fila que Precio que pone por consultar
df_def = df_def[df_def['Precio'] != 'precio a consultar']

#Transformamos la columna de precio a numerico
df_def['Precio'] = pd.to_numeric(df_def['Precio'])

#vemos el tipo de datos de cada columna
df_def.dtypes

Precio                    int64
Descripcion              object
Otras_caractericas       object
Caracteristicas          object
Eficiencia_energetica    object
dtype: object

Tras llamar al dataset y limpiar la columna del precio se va intentar construir un dataframe con columnas de m^2 número de habitaciones, baños etc.

In [48]:
import numpy as np
# Lista para almacenar los resultados
resultados = []

for i in range(len(df_def)):
    # Obtén la información sobre las habitaciones de la fila actual
    habitaciones_info = df_def["Caracteristicas"][i].get("habitaciones")
    
    # Verificar si habitaciones_info no es None
    if habitaciones_info:
        # Buscar el número que acompaña a la palabra "dormitorio"
        dormitorios_numero = re.search(r'(\d+) dormitorios', habitaciones_info)
        if dormitorios_numero:
            dormitorios = dormitorios_numero.group(1)
        else:
            dormitorios = None
    else:
        dormitorios = None
    
    # Agregar el resultado a la lista
    resultados.append(dormitorios)

# Calcular la media de los valores existentes en la lista resultados
media_habitaciones = np.mean([int(x) for x in resultados if x is not None])
media_habitaciones_entero = int(round(media_habitaciones))
# Reemplazar los valores None con la media calculada
resultados_imputados = [media_habitaciones_entero if x is None else x for x in resultados]

# Agregar la lista imputada como una nueva columna al DataFrame
df_def['Habitaciones'] = resultados_imputados

# Mostrar el DataFrame con la nueva columna
print(df_def["Habitaciones"])


https://www.indomio.es/anuncios/95227781/    3
https://www.indomio.es/anuncios/95146207/    4
https://www.indomio.es/anuncios/94613177/    2
https://www.indomio.es/anuncios/94500757/    2
https://www.indomio.es/anuncios/95223135/    3
                                            ..
https://www.indomio.es/anuncios/95007509/    3
https://www.indomio.es/anuncios/94958481/    2
https://www.indomio.es/anuncios/95006805/    2
https://www.indomio.es/anuncios/95313313/    3
https://www.indomio.es/anuncios/95319871/    2
Name: Habitaciones, Length: 1558, dtype: object


C:\Users\alexn\AppData\Local\Temp\ipykernel_5272\680015430.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  habitaciones_info = df_def["Caracteristicas"][i].get("habitaciones")


In [49]:
resultados = []

for i in range(len(df_def)):
    # Obtén la información sobre los baños de la fila actual
    habitaciones_info = df_def["Caracteristicas"][i].get("habitaciones")
    
    # Verificar si habitaciones_info no es None
    if habitaciones_info:
        # Buscar el número que acompaña a la palabra "baño(s)"
        banos_numero = re.search(r'(\d+) baño[s]?', habitaciones_info)
        if banos_numero:
            banos = banos_numero.group(1)
        else:
            banos = None
    else:
        banos = None
    
    # Agregar el resultado a la lista
    resultados.append(banos)

media_banos = np.mean([int(x) for x in resultados if x is not None])
media_banos_entero = int(round(media_banos))
# Reemplazar los valores None con la media calculada
resultados_imputados = [media_banos_entero if x is None else x for x in resultados]

# Agregar la lista imputada como una nueva columna al DataFrame
df_def['Baños'] = resultados_imputados

print(df_def["Baños"])



https://www.indomio.es/anuncios/95227781/    1
https://www.indomio.es/anuncios/95146207/    1
https://www.indomio.es/anuncios/94613177/    1
https://www.indomio.es/anuncios/94500757/    2
https://www.indomio.es/anuncios/95223135/    1
                                            ..
https://www.indomio.es/anuncios/95007509/    1
https://www.indomio.es/anuncios/94958481/    2
https://www.indomio.es/anuncios/95006805/    2
https://www.indomio.es/anuncios/95313313/    1
https://www.indomio.es/anuncios/95319871/    1
Name: Baños, Length: 1558, dtype: object


C:\Users\alexn\AppData\Local\Temp\ipykernel_5272\1073160764.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  habitaciones_info = df_def["Caracteristicas"][i].get("habitaciones")


In [50]:
metros_cuadrados_lista = []
for i in range(len(df_def)):
    # Convertir la entrada de la columna "Descripcion" a cadena de texto
    texto = str(df_def["Descripcion"][i])
    
    # Buscar los metros cuadrados usando una expresión regular
    metros_cuadrados_match = re.search(r'(\d+(?:\.\d+)?)\s*(?:m²|m2|metros\s+cuadrados|metros)', texto)
    
    # Verificar si se encontró la información de los metros cuadrados
    if metros_cuadrados_match:
        metros_cuadrados = float(metros_cuadrados_match.group(1))  # Convertir a float en lugar de int
    else:
        metros_cuadrados = None
    
    # Agregar el resultado a la lista
    metros_cuadrados_lista.append(metros_cuadrados)



media_m2 = np.mean([int(x) for x in metros_cuadrados_lista if x is not None])
media_m2_entero = int(round(media_m2))
# Reemplazar los valores None con la media calculada
resultados_imputados = [media_m2_entero if x is None else x for x in metros_cuadrados_lista]

# Agregar la lista imputada como una nueva columna al DataFrame
df_def['Metros Cuadrados'] = resultados_imputados

print(df_def["Metros Cuadrados"])

https://www.indomio.es/anuncios/95227781/    169.0
https://www.indomio.es/anuncios/95146207/    120.0
https://www.indomio.es/anuncios/94613177/    169.0
https://www.indomio.es/anuncios/94500757/    169.0
https://www.indomio.es/anuncios/95223135/    169.0
                                             ...  
https://www.indomio.es/anuncios/95007509/     95.0
https://www.indomio.es/anuncios/94958481/    169.0
https://www.indomio.es/anuncios/95006805/    169.0
https://www.indomio.es/anuncios/95313313/    169.0
https://www.indomio.es/anuncios/95319871/    169.0
Name: Metros Cuadrados, Length: 1558, dtype: float64


C:\Users\alexn\AppData\Local\Temp\ipykernel_5272\2694596815.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  texto = str(df_def["Descripcion"][i])


In [51]:
# Se crea una lista para almacenar los resultados
resultados = []
for i in range(len(df_def)):
    contrato = df_def["Caracteristicas"][i].get("contrato")
    
    if contrato:
        contrato = contrato.lower()
    else:
        contrato = None
    
    resultados.append(contrato)

df_def["Contrato"] = resultados

print(df_def["Contrato"])

C:\Users\alexn\AppData\Local\Temp\ipykernel_5272\4009006902.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  contrato = df_def["Caracteristicas"][i].get("contrato")


https://www.indomio.es/anuncios/95227781/    alquiler
https://www.indomio.es/anuncios/95146207/    alquiler
https://www.indomio.es/anuncios/94613177/    alquiler
https://www.indomio.es/anuncios/94500757/    alquiler
https://www.indomio.es/anuncios/95223135/    alquiler
                                               ...   
https://www.indomio.es/anuncios/95007509/       venta
https://www.indomio.es/anuncios/94958481/       venta
https://www.indomio.es/anuncios/95006805/       venta
https://www.indomio.es/anuncios/95313313/       venta
https://www.indomio.es/anuncios/95319871/       venta
Name: Contrato, Length: 1558, dtype: object


In [52]:
#Se cambia el tipo de datos de las columnas habitaciones, baños a numerico y el de contrato a stirng

df_def["Habitaciones"] = pd.to_numeric(df_def["Habitaciones"])
df_def["Baños"] = pd.to_numeric(df_def["Baños"])
df_def["Contrato"] = df_def["Contrato"].astype(str)


df_def.dtypes

Precio                     int64
Descripcion               object
Otras_caractericas        object
Caracteristicas           object
Eficiencia_energetica     object
Habitaciones               int64
Baños                      int64
Metros Cuadrados         float64
Contrato                  object
dtype: object

In [53]:
import spacy
import re
import unicodedata
import string

# Cargar el modelo de spaCy para el idioma español
nlp = spacy.load("es_core_news_sm")

def remove_accents(text):
    new_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return new_text

def remove_special_characters(text):
    pat = '[{}]'.format(re.escape(string.punctuation))
    return re.sub(pat, '', text)

def preprocess_text(text):
    if text is None:
        return ''
    # Convertir texto a minúsculas
    text = text.lower()
    # Eliminar acentos
    text = remove_accents(text)
    # Eliminar caracteres especiales y números
    text = remove_special_characters(text)
    # Tokenización y lematización con spaCy
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc]
    # Eliminar stopwords (opcional)
    tokens = [word for word in tokens if not nlp.vocab[word].is_stop]
    return ' '.join(tokens)

# Aplicar la función de preprocesado a la columna 'Descripcion'
df_def['Descripcion'] = df_def['Descripcion'].apply(preprocess_text)

print(df_def['Descripcion'])


https://www.indomio.es/anuncios/95227781/    alquilar piso amueblado 3 habitación carrer de...
https://www.indomio.es/anuncios/95146207/    piso estudiantes entrar 1ro febrero 30 junio p...
https://www.indomio.es/anuncios/94613177/    disponible alquiler mensual anual 2 dormitorio...
https://www.indomio.es/anuncios/94500757/    disponible alquiler mensual anual 2 dormitorio...
https://www.indomio.es/anuncios/95223135/      rincon acogedor   necesitar espacio   gustar...
                                                                   ...                        
https://www.indomio.es/anuncios/95007509/    atico 95 m2 valencia valencia solidinmuebl ven...
https://www.indomio.es/anuncios/94958481/    vender piso obra camino moncada tratar residen...
https://www.indomio.es/anuncios/95006805/    vender exclusiva 6 vivienda obra calle arquite...
https://www.indomio.es/anuncios/95313313/    vender piso barrio tormoseste luminoso apartam...
https://www.indomio.es/anuncios/95319871/    piso 

In [57]:
# Seleccionar las columnas relevantes
columns = ['Descripcion', 'Precio', 'Habitaciones', 'Baños', 'Metros Cuadrados', 'Contrato']
df_filtered = df_def[columns]

min_bedrooms = 2
max_price = 1500
min_bathrooms = 1
min_sqft = 120
contrato = "alquiler"

# List to keep track of applied filters
applied_filters = []

# Filtrar el DataFrame por habitaciones
if min_bedrooms is not None and len(df_filtered[df_filtered['Habitaciones'] >= min_bedrooms]) >= 5:
    df_filtered = df_filtered[df_filtered['Habitaciones'] >= min_bedrooms]
    applied_filters.append('Habitaciones')

# Filtrar el DataFrame por precio
if max_price is not None and len(df_filtered[df_filtered['Precio'] <= max_price]) >= 5:
    df_filtered = df_filtered[df_filtered['Precio'] <= max_price]
    applied_filters.append('Precio')

# Filtrar el DataFrame por baño
if min_bathrooms is not None and len(df_filtered[df_filtered['Baños'] >= min_bathrooms]) >= 5:
    df_filtered = df_filtered[df_filtered['Baños'] >= min_bathrooms]
    applied_filters.append('Baños')

# Filtrar el DataFrame por metros cuadrados
if min_sqft is not None and len(df_filtered[df_filtered['Metros Cuadrados'] >= min_sqft]) >= 5:
    df_filtered = df_filtered[df_filtered['Metros Cuadrados'] >= min_sqft]
    applied_filters.append('Metros cuadrados')

# Filtrar el DataFrame por contrato
if contrato is not None and len(df_filtered[df_filtered['Contrato'] == contrato]) >= 5:
    df_filtered = df_filtered[df_filtered['Contrato'] == contrato]
    applied_filters.append('Contrato')

print("Applied filters:", applied_filters)

Applied filters: ['Habitaciones', 'Precio', 'Baños', 'Metros cuadrados', 'Contrato']


In [58]:
user_description = "Un piso de alquiler en Campanar por menos de 1500 euros al mes, con tres habitaciones, dos baños y terraza. Uno de los baños tiene bañera y el otro ducha. El salón debe ser espacioso y luminoso y la cocina debe ser moderna y estar amueblada"
user_desc_processed = [preprocess_text(user_description)]
print(user_desc_processed)

['piso alquiler campanar 1500 euros mes habitación bano terrazar bano banera ducha salon deber espacioso luminoso cocina deber moderno amueblado']


### BOW

In [59]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

cv = CountVectorizer()
user_desc_bow = cv.fit_transform(user_desc_processed)
desc_bow = cv.transform(df_filtered['Descripcion'])
vocab = cv.get_feature_names_out()

similarity_bow = cosine_similarity(user_desc_bow, desc_bow)

print(vocab)

['1500' 'alquiler' 'amueblado' 'banera' 'bano' 'campanar' 'cocina' 'deber'
 'ducha' 'espacioso' 'euros' 'habitación' 'luminoso' 'mes' 'moderno'
 'piso' 'salon' 'terrazar']


In [60]:
import numpy as np

sorted_indices = np.argsort(similarity_bow[0,:])

filtered_df_bow = df_filtered.iloc[sorted_indices]

print(filtered_df_bow)

                                                                                 Descripcion  \
https://www.indomio.es/anuncios/95041839/  habitacion persona precio economico persona 36...   
https://www.indomio.es/anuncios/95223135/    rincon acogedor   necesitar espacio   gustar...   
https://www.indomio.es/anuncios/95183841/                                                      
https://www.indomio.es/anuncios/95024555/  excavación centro amar vecindario vibrante   g...   
https://www.indomio.es/anuncios/94845913/  alquilar departamento amoblado 2 recamara av p...   
...                                                                                      ...   
https://www.indomio.es/anuncios/94984795/  piso alquiler actualmente oficina estancia ban...   
https://www.indomio.es/anuncios/94798533/  estupendo luminoso apartamento exterior situad...   
https://www.indomio.es/anuncios/95155457/  engel   volkers presentar luminoso piso estren...   
https://www.indomio.es/anuncios/95312521

### TFIDF

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer()
user_desc_tfidf = tv.fit_transform(user_desc_processed)
desc_tfidf = tv.transform(df_filtered['Descripcion'])
vocab = tv.get_feature_names_out()

similarity_tfidf = cosine_similarity(user_desc_tfidf, desc_tfidf)

print(vocab)

['1500' 'alquiler' 'amueblado' 'banera' 'bano' 'campanar' 'cocina' 'deber'
 'ducha' 'espacioso' 'euros' 'habitación' 'luminoso' 'mes' 'moderno'
 'piso' 'salon' 'terrazar']


In [62]:
import numpy as np

sorted_indices = np.argsort(similarity_tfidf[0,:])

filtered_df_tfidf = df_filtered.iloc[sorted_indices]

print(filtered_df_tfidf)

                                                                                 Descripcion  \
https://www.indomio.es/anuncios/95041839/  habitacion persona precio economico persona 36...   
https://www.indomio.es/anuncios/95283809/    juerguista rustico   querer encantado   gust...   
https://www.indomio.es/anuncios/95024555/  excavación centro amar vecindario vibrante   g...   
https://www.indomio.es/anuncios/95048745/                                                      
https://www.indomio.es/anuncios/95315049/  alquilar departamento amoblado 3 recamara av g...   
...                                                                                      ...   
https://www.indomio.es/anuncios/94798533/  estupendo luminoso apartamento exterior situad...   
https://www.indomio.es/anuncios/94984795/  piso alquiler actualmente oficina estancia ban...   
https://www.indomio.es/anuncios/95155457/  engel   volkers presentar luminoso piso estren...   
https://www.indomio.es/anuncios/95312521